In [ ]:
import pandas as pd

# Try reading the file with a different encoding
df = pd.read_csv("/kaggle/input/dataco-supply-chain-dataset-description/DescriptionDataCoSupplyChain (1).csv", encoding='ISO-8859-1')
df

In [ ]:
import pandas as pd

# Try reading the file with a different encoding
df = pd.read_csv("/kaggle/input/dataco-supply-chain-dataset/DataCoSupplyChainDataset (1).csv", encoding='ISO-8859-1')

# Display basic information about the dataset
df.info()


#df = pd.read_csv("/kaggle/input/dataco-supply-chain-dataset/DataCoSupplyChainDataset (1).csv")

In [ ]:
df.head()

In [ ]:
df.columns.values

## Data Cleaning

In [ ]:

# Check for missing values
missing_values = df.isnull().sum().sort_values(ascending=False)

# Check for duplicate rows
duplicates = df.duplicated().sum()

# Display columns with missing values and number of duplicate rows
missing_values[missing_values > 0], duplicates

### Missing Data:

* Customer Lname : 8

* Customer Zipcode : 3

* Order Zipcode : 155679 has significant missing data (only ~13% non-null) ==>

* Product Description is entirely missing. ==>> remove this feature

#### Possible Actions:

* Drop columns with excessive or complete missing data: Product Description, Order Zipcode.

* Leave or impute minor missing values if needed.

* Ensure date formats are correct (already converted).

In [ ]:
import pandas as pd

# Reload the dataset
file_path = '/kaggle/input/dataco-supply-chain-dataset/DataCoSupplyChainDataset (1).csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Drop columns with excessive missing data
df = df.drop(columns=['Product Description', 'Order Zipcode'])

# Convert date columns to datetime
df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'], errors='coerce')
df['shipping date (DateOrders)'] = pd.to_datetime(df['shipping date (DateOrders)'], errors='coerce')

In [ ]:

# Check for missing values
missing_values = df.isnull().sum().sort_values(ascending=False)

# Check for duplicate rows
duplicates = df.duplicated().sum()

# Display columns with missing values and number of duplicate rows
missing_values[missing_values > 0], duplicates

In [ ]:
!pip install ace_tools

In [ ]:
# Generate summary statistics for all columns
summary_stats = df.describe(include='all').transpose()

# # Display the summary statistics
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Summary Statistics", dataframe=summary_stats)

summary_stats

In [ ]:
import matplotlib.pyplot as plt

# Select relevant numerical features for distribution analysis
numerical_cols = [
    'Days for shipping (real)', 'Days for shipment (scheduled)',
    'Benefit per order', 'Sales per customer', 'Order Item Discount',
    'Order Item Discount Rate', 'Order Item Product Price',
    'Order Item Profit Ratio', 'Order Item Quantity', 'Sales',
    'Order Item Total', 'Order Profit Per Order', 'Product Price'
]

# Plot histograms for each numerical column
for col in numerical_cols:
    plt.figure()
    df[col].hist(bins=50)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.grid(False)
    plt.tight_layout()
    plt.show()



* Days for shipping (real) is slightly right-skewed; most shipments are within 2–5 days.

* Days for shipment (scheduled) is tightly packed around 4 days.

* Benefit per order and Order Profit Per Order show a long right tail and include some negative values, indicating occasional losses.

* Sales per customer and Sales are right-skewed – a small number of customers generate high sales.

* Order Item Discount and Discount Rate are heavily skewed, mostly near zero, with a few high-discount cases.

* Product Price and Order Item Product Price distributions are skewed with a majority of items in lower price ranges.

* Order Item Profit Ratio shows that while most items are profitable, a subset has unusually high or low profit margins.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute correlation matrix for numerical features
correlation_matrix = df[numerical_cols].corr()

# Plot the correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation Heatmap of Numerical Features")
plt.tight_layout()
plt.show()

* Sales and Order Item Total (0.99): Indicates total sales are directly driven by order totals.

* Sales per customer and Benefit per order (0.95): Suggests high-spending customers are more profitable.

* Order Item Product Price and Product Price (0.89): Product-level and item-level prices align closely.

* Order Item Quantity negatively correlates with Order Item Product Price and Product Price: Possibly due to bulk discounts or cheaper items being purchased in higher quantities.

* Order Profit Per Order positively correlates with Benefit per order and Sales.

* Shipping days (real and scheduled) have low correlation with financial metrics, suggesting logistics might not directly affect revenue or profit in this dataset.